In [1]:
import pandas as pd
from selenium import webdriver
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
df_recetas = pd.read_csv('recetas.csv', delimiter='|')

In [3]:
df_recetas = df_recetas.dropna(subset=['Tiempo'])

In [4]:
df_recetas = df_recetas.dropna(subset=['Tipo'])

In [5]:
df_recetas = df_recetas.dropna(subset=['Ingredientes'])

In [6]:
df_recetas = df_recetas.dropna(subset=['Dificultad'])


In [7]:
df_recetas = df_recetas.drop(columns = ['Num_comentarios', 'Num_reviews', 'Fecha_modificacion'])

In [8]:
df_recetas.head()

,Id,Categoria,Nombre,Valoracion,Dificultad,Num_comensales,Tiempo,Tipo,Link_receta,Ingredientes
1,74133,Recetas de Aperitivos y tapas,Receta de Tacos de carne molida,NaN,baja,4.0,30m,Plato principal,https://www.recetasgratis.net/receta-de-tacos-...,"500 gramos de carne molida,1 papa mediana,2 cu..."
2,74154,Recetas de Aperitivos y tapas,Receta de Burritos norteños,NaN,muy baja,5.0,30m,Entrante,https://www.recetasgratis.net/receta-de-burrit...,"10 tortillas de harina para burritos,500 gramo..."
3,74146,Recetas de Aperitivos y tapas,Receta de Croquetas de pollo caseras de la abuela,NaN,baja,4.0,45m,Entrante,https://www.recetasgratis.net/receta-de-croque...,"1 pechuga de pollo cocida y troceada,1 pellizc..."
4,74081,Recetas de Aperitivos y tapas,Receta de Masa para empanadas fritas,NaN,baja,3.0,30m,Entrante,https://www.recetasgratis.net/receta-de-masa-p...,"3 tazas de harina de trigo (420 gramos),1 pizc..."
5,74068,Recetas de Aperitivos y tapas,Receta de Hummus de zanahoria,NaN,baja,6.0,1h 30m,Entrante,https://www.recetasgratis.net/receta-de-hummus...,"2 zanahorias,1 chorro de aceite de olivia virg..."


# Scrapeo

In [9]:
driver = 'chromedriver.exe'
url = 'https://www.recetasgratis.net/'

In [10]:
options = webdriver.ChromeOptions()
options.add_argument("headless");
driver = webdriver.Chrome(executable_path=driver, options=options)

In [11]:
driver.get(url)

In [12]:
button = driver.find_element_by_xpath('/html/body/div[7]/div[2]/div[1]/div[2]/div[2]/button[1]/p')

In [13]:
button.click()

In [14]:
import time
tt = time.time()
lista_id = []
pasos = []
ins = []
categorias = []
num = -1
for url in tqdm(df_recetas['Link_receta']):
    driver.get(url)
    num += 1
    for i in driver.find_elements_by_class_name('apartado'):
        if i.find_elements_by_class_name('orden') != []:
            try:
                lista_id.append(df_recetas['Id'].iloc[[num]].values)
                pasos.append(int(i.find_element_by_tag_name('div').text))
                ins.append(i.find_element_by_tag_name('p').text)
                categorias.append(driver.find_element_by_xpath('/html/body/div/div[3]/div[2]/article/div[2]/div[4]/div[2]').text)
            except:
                continue

df_pasos = pd.DataFrame(list(zip(lista_id, pasos, ins, categorias)),
               columns =['Id', 'Paso', 'Instruccion', 'Categorias'])

df_pasos['Categorias'] = df_pasos['Categorias'].map(lambda x: x.lstrip('Características adicionales:'))

df_pasos.to_csv(r'./df_pasos.csv', index=False)

tt - time.time()

  0%|          | 0/11370 [00:00<?, ?it/s]

-7950.145122289658

In [15]:
palabros = []

for i in df_pasos['Categorias'].str.split(','):
    for j in i:
        if j not in palabros:
            palabros.append(j)

In [16]:
palabros

['barato',
 ' Picante',
 ' Recetas mexicanas',
 ' Popular en Primavera-Verano',
 ' Poco picante',
 ' Ideal acompañar con cerveza',
 ' En sartén',
 ' Receta sin azúcar',
 ' Frito',
 ' Recetas chilenas',
 ' Recomendada para vegetarianos',
 ' Recetas al horno',
 ' Receta tradicional de Egipto',
 ' Confitado',
 'medio',
 ' Marinado',
 ' Recomendada para veganos',
 ' Recomendada para macrobióticos',
 ' Recomendada para perder peso',
 ' Receta sin lactosa',
 ' Hervido',
 ' Al grill',
 ' Receta tradicional de Siria',
 ' Popular en Otoño-Invierno',
 ' Nada picante',
 ' Sin cocción',
 ' Receta sin gluten',
 ' Recetas al vapor',
 ' Receta tradicional de Rumanía',
 ' En frío',
 ' Recetas españolas',
 ' Recetas andaluzas',
 ' Ideal acompañar con zumo natural',
 ' Receta tradicional de Alemania',
 ' Recetas colombianas',
 ' Recetas gallegas',
 ' Salteado',
 ' Recetas para Navidad',
 ' Recetas venezolanas',
 ' Estofado',
 ' Receta tradicional de Paraguay',
 ' Plancha',
 ' Receta tradicional de Etiop